In [ ]:
# import all from /raw_data/

In [ ]:
# find maximums of accelerometer along the vertical axis, take N samples aruond those maximums
# throw away the samples not taken

In [ ]:
# plot the distributions of each feature and descriptive statistics

In [ ]:
# normalize the data?
# transformations?

In [ ]:
# output to /processed_data/